# 基础篇
## 01 Heap/Stack/RAII
> 知识点： 
> * new/delete的底层实现为malloc/free   
> * 当出现异常时，会发生栈展开(stack unwinding). 栈上变量的析构函数会被调用，所以即使有异常，也不用担心栈上的内存泄露。
> * 使用派生类赋值于一个基类，必须使用指针。如果直接赋值，则会出现对象切片现象(object slicing), 也就是说，派生类的数据丢失了.

虽然栈不用担心内存泄露，但在栈上将对象实例化并不一定是一个好的选择，因为对象往往非常占用空间而栈的空间又非常珍贵。当然，也不要走极端，在栈上还是堆上建立实例取决于具体的使用场景。

## 02 智能指针
还是不能过分神话智能指针，因为说到底它还是指针而已，只不过多了一些功能来帮助我们管理指针的生命周期。
大体上，智能指针的可以提供如下功能：
* 析构时自动释放内存 (基于RAII)
* 支持运算符*解引用
* 支持运算符->指向对象成员
* 支持布尔表达式
* 线程安全
* 支持指针类型转换

如果是unique_ptr，则会：
* 禁止赋值构造
* 仅允许移动构造

如果是shared_ptr，则会：
* 支持计数器
 * 赋值构造时，计数器加一
 * 析构时，计数器减一
 * 计数器为0时，释放指针内存
* 支持自循环

In [1]:
#include <utility>

template<typename T>
class smart_ptr
{
public:
    explicit smart_ptr(T* ptr = nullptr){ptr_ = ptr;}
    ~smart_ptr(){delete ptr_;}
    
    T& operator*() const {return *ptr_;}
    T* operator->() const {return ptr_;}
    operator bool() const {return (ptr_ != nullptr);}
    
    smart_ptr(smart_ptr&& other){
        ptr_ = other.release();
    }
    
    T& operator=(smart_ptr other){
        other.swap(*this);
        return *this;
    }
    
    // 这个不算是移动构造函数
    // 它可以帮助我们实现基类和派生类智能指针之间的赋值
    template <typename U>
    smart_ptr(smart_ptr<U>&& other)
    {
        ptr_ = other.release();
    }
    
    // Release object
    T* release(){
        T* ptr = ptr_;
        ptr_ = nullptr;
        return ptr;
    }
    
    void swap(smart_ptr& other){
        std::swap(ptr_, other.ptr_);
    }
    
private:
    T* ptr_;
};

> 知识点： 如果提供了移动构造函数，而没有手动提供赋值构造函数，则后者会被禁用。